In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import os
import requests

In [23]:
load_dotenv("../venv/secure/tokens.env")
CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIPY_CLIENT_SECRET")
REDIRECT_URI = os.getenv("SPOTIPY_REDIRECT_URI")
scope = "playlist-read-private user-library-read"
#
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope=scope,
    requests_timeout=30
))

In [24]:
try:
    user = sp.current_user()
    print("Connected! Username:", user['display_name'])
except Exception as e:
    print("Connection failed:", e)

Connection failed: [Errno 48] Address already in use


In [25]:
LASTFM_API_KEY = os.getenv("LASTFM_API_KEY")
BASE_URL = "http://ws.audioscrobbler.com/2.0/"
params = {
        "method": "artist.getinfo",
        "artist": "Radiohead",
        "api_key": LASTFM_API_KEY,
        "format": "json"
    }
response = requests.get(BASE_URL, params=params)

if response.status_code == 200:
    data = response.json()
    if "artist" in data:
        print("Connected: Artist:", data["artist"]["name"])
    else:
        print("Connected: Artists not found. Response keys:", data.keys())
else:
    print("Failed to connect. Status code:", response.status_code, "Response:", response.text)

Connected: Artist: Radiohead


In [26]:
import requests
from xml.etree import ElementTree as ET

# 🔑 Replace with your Last.fm API key
LASTFM_API_KEY = os.getenv("LASTFM_API_KEY")
ARTIST_NAME = "Adele"

def get_artist_info(artist_name, api_key):
    url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getInfo&artist={artist_name}&api_key={api_key}&format=xml"
    try:
        res = requests.get(url, timeout=10)
        res.raise_for_status()
        root = ET.fromstring(res.text)
        
        # Fetch plays and listeners
        stats = root.find(".//stats")
        listeners = int(stats.find("listeners").text) if stats is not None and stats.find("listeners") is not None else 0
        # Fetch up to 3 genres/tags
        tags_root = root.findall(".//tags/tag")
        genres = [tag.find("name").text for tag in tags_root[:3]] if tags_root else []

        return {"artist": artist_name, "listeners": listeners, "genres": genres}
    
    except Exception as e:
        print(f"Error fetching info for {artist_name}: {e}")
        return {"artist": artist_name, "plays": 0, "listeners": 0, "genres": []}

info = get_artist_info(ARTIST_NAME, LASTFM_API_KEY)
print(info)

{'artist': 'Adele', 'listeners': 5368134, 'genres': ['soul', 'female vocalists', 'british']}
